# Initial experiments Part 2

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from dataset import QuickDrawDataset
from utils import DeltaPenPositionTokenizer
from prepare_data import stroke_to_rdp, stroke_to_bezier_single, clean_svg
from tqdm import tqdm
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

seed = 42
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)

Using device: cuda


In [3]:
labels = ["bird", "crab", "guitar"]
training_data = QuickDrawDataset(labels=labels, download=True)
tokenizer = DeltaPenPositionTokenizer(bins=32)


class SketchDataset(Dataset):
    def __init__(
        self,
        svg_list,
        tokenizer,
        max_len=200,
        cache_file="sketch_multi_dataset.pkl",
    ):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_id = tokenizer.vocab["PAD"]
        self.parent = svg_list
        self.num_labels = len(svg_list.label_map)

        # Try to load from cache
        try:
            with open(cache_file, "rb") as f:
                self.data = pickle.load(f)
            print(f"Loaded tokenized data from {cache_file}")
        except FileNotFoundError:
            for svg in tqdm(svg_list, desc="Tokenizing SVGs"):
                # use RDP to reduce number of points in SVG (already applied for quickdraw)
                # svg = stroke_to_rdp(svg, epsilon=1.0)  # tuning
                tokens = tokenizer.encode(svg)[:max_len]
                tokens = tokens + [self.pad_id] * (max_len - len(tokens))
                self.data.append(tokens)

            with open(cache_file, "wb") as f:
                pickle.dump(self.data, f)
            print(f"Saved tokenized data to {cache_file}")

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq[:-1])
        target_ids = torch.tensor(seq[1:])
        return input_ids, target_ids, self.parent.labels[idx]  # integer

    def __len__(self):
        return len(self.data)


dataset = SketchDataset(training_data, tokenizer)

Loading QuickDraw files: 3it [00:09,  3.07s/it]
Tokenizing SVGs: 100%|██████████| 329852/329852 [01:47<00:00, 3064.59it/s]


Saved tokenized data to sketch_multi_dataset.pkl


In [5]:
def generate_square_subsequent_mask(sz: int):
    """Causal mask to stop attention to future positions"""
    return torch.triu(torch.ones(sz, sz), diagonal=1).bool()


class SketchTransformer(nn.Module):
    def __init__(
        self, vocab_size, num_classes, d_model=512, nhead=8, num_layers=6, max_len=200
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len

        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)
        self.class_embed = nn.Embedding(num_classes, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4 * d_model
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x, class_labels):
        """
        x: (batch, seq_len) input tokens
        class_labels: (batch,) integer labels for conditioning
        Returns: (batch, seq_len, vocab_size)
        """
        batch_size, seq_len = x.shape
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        x = self.embed(x) + self.pos_embed(positions)
        class_cond = self.class_embed(class_labels).unsqueeze(1)  # (batch, 1, d_model)
        x = x + class_cond  # simple additive conditioning
        x = x.transpose(0, 1)  # (seq_len, batch, d_model)
        mask = generate_square_subsequent_mask(seq_len).to(x.device)
        x = self.transformer(x, mask=mask)
        x = x.transpose(0, 1)  # back to (batch, seq_len, d_model)

        logits = self.fc_out(x)
        return logits


def train_model(model, dataloader, vocab_size, epochs=10, lr=1e-4, device="cuda"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, target_ids, class_labels in tqdm(
            dataloader, desc=f"Epoch {epoch+1}/{epochs}"
        ):
            input_ids, target_ids, class_labels = (
                input_ids.to(device),
                target_ids.to(device),
                class_labels.to(device),
            )

            logits = model(input_ids, class_labels)
            loss = criterion(logits.view(-1, vocab_size), target_ids.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")


dataloader = DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)
model = SketchTransformer(
    vocab_size=len(tokenizer.vocab),
    num_classes=dataset.num_labels,
    d_model=512,
    nhead=8,
    num_layers=8,
)

# d_model => model capacity (types of drawing features it can learn)
# nhead => model can attend to more positions in parallel
# num layers => model learns more hierarchical abstractions (patterns, shapes , layouts)

train_model(
    model,
    dataloader,
    vocab_size=len(tokenizer.vocab),
    epochs=15,
    lr=1e-4,
    device=device,
)

c:\Users\matth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Epoch 1/15: 100%|██████████| 2577/2577 [14:29<00:00,  2.97it/s]


Epoch 1 Loss: 1.2429


Epoch 2/15: 100%|██████████| 2577/2577 [14:22<00:00,  2.99it/s]


Epoch 2 Loss: 1.0938


Epoch 3/15: 100%|██████████| 2577/2577 [14:35<00:00,  2.94it/s]


...

Epoch 8 Loss: 0.9939


Epoch 9/15: 100%|██████████| 2577/2577 [14:15<00:00,  3.01it/s]


Epoch 9 Loss: 0.9883


Epoch 10/15:  72%|███████▏  | 1867/2577 [10:20<03:55,  3.01it/s]


KeyboardInterrupt: 

In [6]:
torch.save(model, "sketch_transformer_model_cond_v2_deep.pth")
# model = torch.load(
#     "sketch_transformer_model_cond_v2_deep.pth", map_location=device, weights_only=False
# )

In [8]:
def top_p_filtering(logits, p=0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = False
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[:, indices_to_remove] = -float("Inf")
    return logits


def top_k_filtering(logits, k):
    if k <= 0:
        return logits
    top_k = min(k, logits.size(-1))
    values, _ = torch.topk(logits, top_k)
    min_values = values[:, -1].unsqueeze(-1)
    logits[logits < min_values] = -float("Inf")
    return logits


def sample_sequence_feat(
    model,
    start_tokens,
    class_label,
    max_len=200,
    temperature=0.8,
    top_k=20,
    top_p=0.7,
    greedy=False,
    eos_id=None,
    device="cuda",
):
    model.eval()

    tokens = list(start_tokens)
    tokens_tensor = torch.tensor([tokens], device=device, dtype=torch.long)
    class_label_tensor = torch.tensor([class_label], device=device, dtype=torch.long)

    for _ in range(max_len - len(tokens)):
        with torch.no_grad():
            logits = model(tokens_tensor, class_label_tensor)
            next_logits = logits[:, -1, :] / temperature

            # top-k / top-p filtering
            next_logits = top_k_filtering(next_logits, top_k)
            next_logits = top_p_filtering(next_logits, top_p)

            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)
        if eos_id is not None and next_token == eos_id:
            break

        # Append new token for next iteration
        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens


start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)
generations_inline = ""
generations = []

labels = ["bird", "crab", "guitar"]

for j, label in enumerate(labels):
    for i in range(5):
        generated = sample_sequence_feat(
            model,
            start_tokens=[start_token],
            class_label=j,
            max_len=200,
            greedy=False,
            eos_id=eos_token,
            device=device,
        )
        decoded_sketch = tokenizer.decode(generated, stroke_width=0.3)

        decoded_sketch = stroke_to_bezier_single(decoded_sketch)
        decoded_sketch = clean_svg(decoded_sketch)

        # print("Generated token sequence:", generated)
        # print("Decoded sketch:", decoded_sketch)
        generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated {label}</b><br>{decoded_sketch}</div>'
        generations.append((generated, decoded_sketch))


from IPython.display import HTML, display

display(HTML(generations_inline))

# temp<=0.5 fairly deterministic

# temp=0.8, top_k=20, top_p=0.9 more variety but still coherent

# *note important features are usually preseved, but sketches are disorganized (number of curves hueristic does not work well)*
# temp=1.0, top_k=20, top_p=0.75 more variety, some incoherent sequences

#  *note that lower temp means less variety, notice that sequences begin to repete themselves more often*
# temp=0.55, top_k=20, top_p=0.9 good balance
# temp=0.6, top_k=30, top_p=0.9  good balance

In [ ]:
# note many sketches have missing parts or incomplete shapes (step 1: get a base sketch) : check the number of paths
# psuedo hueuristic: count number of curves in SVG

from prepare_data import count_curves

# sort generations by number of curves
generations_inline = ""

generations_sorted = sorted(generations, key=lambda x: count_curves(x[1]), reverse=True)
for sketch in generations_sorted:
    generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{sketch[1]}</div>'

display(HTML(generations_inline))